# Code breakdown (por melhorar)

A implementação deste projeto é fortemente dependente das bibliotecas "astropy" e "numpy". Recorrendo a estas de forma a poder aceder e manipular ficheiros fits e facilitar a execução de cálculos de matrizes, respetivamente.

As bibliotecas "os" e "sys", "matplotlib.pyplot" e "glob" são bibliotecas auxiliares. 

In [2]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists
import sys
import glob

A função "name_gen" é utilizada para gerar um nome único para um novo ficheiro a ser gerado.

In [6]:
def name_gen(file_name):
    i=0
    check_name = file_name
    while exists(check_name):
        i+=1
        check_name = file_name[:-5]+"_new"+str(i)+".fits"
    print(f"Novo ficheiro gerado: {check_name}")
    return check_name

In [7]:
name_gen("rascunho.fits")

Novo ficheiro gerado: rascunho_new1.fits


'rascunho_new1.fits'

A seguinte função é crítica para o funcionamento do programa, servindo como função auxiliar da função principal "remove_hotpixels".

A função "argwhere" da biblioteca "numpy" cria uma lol (list of lists) que contem as coordenadas de todos os elementos de um lista/lol que respeitam uma determinada condição. Neste caso, argwhere retorna uma lol com as coordenadas de todos os elementos da matriz de dados de uma HDU com valor superior a limit.

No contexto deste programa esta função tem como objetivo mapear os pixeis quentes (dai o seu nome) numa imagem fits.

In [ ]:
def get_hotpixels(hdul, limit):
    data = hdul["PRIMARY"].data
    return np.argwhere(data >= limit)

In [3]:
#EXEMPLO
data = [[1,5,3,6,11],
        [2,4,2,3,3],
        [19,0,-3, 4,4]]
data = np.array(data)

coords = np.argwhere(data >= 5)
print("Coordenadas dos pontos quentes:")
print(coords)
print("\n")
print("Valores dos pontos quentes:")
for i,j in coords:
    print(data[i][j])

Coordenadas dos pontos quentes:
[[0 1]
 [0 3]
 [0 4]
 [2 0]]


Valores dos pontos quentes:
5
6
11
19


A função "remove_hotpixels" é responsável por trocar o valor dos pixeis quentes pela mediana dos valores vizinhos.
De maneira a alcançar este objetivo ela utiliza os resultados obtidos pela função auxiliar "get_hotpixels".
Com a nova lol é criado um novo ficheiro fits onde os pontos quentes mapeados foram alterados.

In [ ]:
def remove_hotpixels(hdu, hot_pixels, raio=1):
    a = np.copy(hdu["PRIMARY"].data)
    x_res,y_res = a.shape
    for k in hot_pixels:
        line = k[0]
        collumn = k[1]
        vizinhos = []
        for i in range(-raio, raio+1):
            for j in range(-raio, raio+1):
                    if ((line+i>=0 and collumn+j>=0 and line+i<x_res and collumn+j<y_res) and (not(i==0 and j==0))):
                        vizinhos.append(a[line+i][collumn+j])

        vizinhos = np.array(vizinhos)
        mediana = np.median(vizinhos)

        a[line][collumn] = mediana

    file_name = hdu.filename()
    new_name = name_gen(file_name)

    try:
        hdul = fits.PrimaryHDU(a)
        hdul.writeto(new_name,overwrite=True)

    except Exception as e:
        print(e)

"get_filtered_image_data" é utilizada para obter a lol com os valores dos pontos quentes alterados para mediana dos valores vizinhos. Esta função é auxiliar à função seguinte.

In [ ]:
def get_filtered_image_data(hdu_data, hot_pixels, raio):
    a = np.copy(hdu_data)     #Cria uma cópia dos valores da imagem
    x_res,y_res = a.shape     #Dimensões do aray
    for k in hot_pixels:
        line = k[0]
        collumn = k[1]
        vizinhos = []         #Inicializa um array vazio para representar a vizinhança.
        #Percorrre a vizinhança
        for i in range(-raio, raio+1):
            for j in range(-raio, raio+1):
                    #Estas condições lógicas garantem que não se sai do alcança da lol e que o próprio pixel quente não é tratado como seu vizinho
                    if ((line+i>=0 and collumn+j>=0 and line+i<x_res and collumn+j<y_res) and (not(i==0 and j==0))):
                        vizinhos.append(a[line+i][collumn+j]) #Caso faça parte da vizinhança acrescenta à lista desta

        vizinhos = np.array(vizinhos) #transforma num numpy array
        mediana = np.median(vizinhos) #calcula a mediana dos valores vizinhos

        a[line][collumn] = mediana #substitui o valor do pixel quente pela mediana dos valores vizinhos

    return a

A função "get_ADA_HV" é uma função axiliar de "plot_ADA" que devolve a tupla (médias das diferenças absolutas, valores dos pixeis quentes). Estes valores são depois utilizados na função apresentada após esta.

In [ ]:
def get_ADA_HV(hdu, hot_pixels, raio):
    a = np.copy(hdu["PRIMARY"].data)    #Cria uma cópia dos valores da imagem
    abs_diff_average = []               #Inicializa uma lista vazia para armazenar as médias das das diferenças absolutas
    hotpixels_values = []               #Inicializa uma lista vazia para armazenar os valores dos pixeis quentes
    for k in hot_pixels:
        line = k[0]
        collumn = k[1]
        hotpixels_values.append(a[line][collumn]) #Adiciona o valor do pixel quente à lista que armazena os valores dos pixeis quentes
        total = 0
        n_vizinhos = 0
        #Precorre a vizinhança
        for i in range(-raio, raio+1):
            for j in range(-raio, raio+1):
                    #Estas condições lógicas garantem que não se sai do alcança da lol e que o próprio pixel quente não é tratado como seu vizinho
                    if ((line+i>=0 and collumn+j>=0 and line+i<len(a) and collumn+j<len(a[0])) and (not(i==0 and j==0))):
                        total += abs(a[line][collumn] - a[line+i][collumn+j]) #Calcula a diferença absoluta
                        n_vizinhos += 1 #Monotoriza o número dos vizinhos
        abs_diff_average.append(total/n_vizinhos) #Calcula a média das diferenças absolutas da vizinhança

    return abs_diff_average, hotpixels_values

Esta função axiliar à função "plot_ADA" cria o scatter plot para representar gráficamente a relação entre os valores dos pontos quentes e a média da diferença absoluta entre os pontos quentes e os seus vizinhos.

In [ ]:
def scatter_plot(hotpixels_values,average_abs_diff, k):
    plt.scatter(hotpixels_values,average_abs_diff)
    plt.xlabel('Hot values')
    plt.ylabel('Absolute difference average')
    plt.title("K = {}".format(k))
    plt.show()

Função principal que combina as auxiliares "get_ADA_HV" e "scatter_plot" para garantir o bom funcionamento da representação gráfica da relação entre os valores dos pontos quentes e a média da diferença absoluta entre estes e os seus vizinhos.

In [ ]:
def plot_ADA(hdu, darkframe, multiplicadores= [0, 0.5, 1, 1.5, 2, 2.5, 3],raio=1):
    for i in multiplicadores:
        #calcular hot_pixels
        darkframe_copy = np.copy(darkframe["PRIMARY"].data)
        limit = np.mean(darkframe_copy) + i*np.std(darkframe_copy)
        hot_pixels = get_hotpixels(darkframe, limit)

        avg, hot_values = get_ADA_HV(hdu, hot_pixels, raio) #Obtem os AVG e os valores dos hot_pixels para cada multiplicador
        scatter_plot(hot_values,avg, i)

Esta função utiliza "get_filtered_image_data" de forma a criar varias imagens alteradas de forma as utilizar para fazer um "stacking", que se trata de fazer a média dos dados das lol das imagens.

In [ ]:
def get_stacked_image(darkframe, imgs_direct, std_mult=2, raio=1):
    print(f"Multiplicador de desvio = {std_mult}")
    print(f"Raio = {raio}")
    #calcular hot_pixels
    darkframe_data = np.copy(darkframe["PRIMARY"].data)
    limit = np.mean(darkframe_data) + std_mult*np.std(darkframe_data)
    hot_pixels = get_hotpixels(darkframe, limit)

    imgs = glob.glob(imgs_direct+"*.fits") #Obter lista com o path das imagens
    final_image = np.zeros(shape=darkframe_data.shape, dtype=darkframe_data.dtype) #Inicializa o np.array que vai acumular os valores das várias imagens
    for img in imgs:
        with fits.open(img) as image:
            print(f"A processar {img}...")
            final_image += get_filtered_image_data(image["PRIMARY"].data, hot_pixels, raio) #Adicionar a final_image os valores de cada imagem filtrada
    final_image = final_image//len(imgs) #Calcular a media de todas as imagens filtradas
    #Guardar os dados num ficherio fits
    hdu = fits.PrimaryHDU(final_image)
    hdu.writeto(imgs_direct+"stacked.fits", overwrite=True)

O objetivo desta função é explorar uma forma de, a partir de uma lista inicial de possíveis pontos quentes, de acordo com um determinado critério, identificar os pixeis com maior probabilidade de serem um ponto quente.

In [ ]:
def get_true_hotpixels(hdul, hot_pixels, media, desvio, raio=1):
    a = np.copy(hdul["PRIMARY"].data)
    true_hotpixels = []
    for k in hot_pixels:
        line = k[0]
        collumn = k[1]
        total = 0
        n_vizinhos = 0
        for i in range(-raio, raio+1):
            for j in range(-raio, raio+1):
                    if ((line+i>=0 and collumn+j>=0 and line+i<len(a) and collumn+j<len(a[0])) and (not(i==0 and j==0))):
                        total += abs(a[line][collumn] - a[line+i][collumn+j])
                        n_vizinhos += 1
        ADA = total/n_vizinhos
        if ADA >= (media - desvio):
            true_hotpixels.append([line,collumn])

    true_hotpixels = np.array(true_hotpixels)

    return true_hotpixels

Aplica "gamma correction" a uma imagem.

In [ ]:
def gammaCorrection(hdu, gamma=0.4):
    data = np.copy(hdu["PRIMARY"].data)
    for line in data:
        for pixel in line:
            pixel = data.max() * (pixel / data.max()) ** (1/gamma)
    return data

Aplica "Histogram spreading" a uma imagem.

In [ ]:
def histogramSpread(hdu, blackPoint=0, whitePoint=65535):
    data = np.copy(hdu["PRIMARY"].data)
    for line in data:
        for pixel in line:
            if (pixel < blackPoint):
                z = 0
            elif (pixel > whitePoint):
                z = 1
            else:
                z = (pixel - blackPoint)  / (whitePoint - pixel)
            pixel = z * data.max()
    return data

# Exemplos de utilização

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists
import sys
import glob
import functions        

In [ ]:
#Remoção de pixeis quentes de uma imagem 
with fits.open(sys.argv[1]) as darkframe:
    with fits.open(sys.argv[2]) as hdul:
            darkframe_data = np.copy(darkframe["PRIMARY"].data)
            limit = np.mean(darkframe_data) + 2*np.std(darkframe_data)
            hot_pixels = get_hotpixels(darkframe, limit)
            functions.remove_hotpixels(hdul, hot_pixels, raio=2)

In [ ]:
#Representação gráfica da relação entre os valores dos pontos quentes e a média da diferença absoluta entre os pontos quentes e os seus vizinhos.
with fits.open(sys.argv[1]) as darkframe:
        with fits.open(sys.argv[2]) as hdu:
            functions.plot_ADA(hdu, darkframe, multiplicadores=[0, 0.5, 1, 1.5, 2, 2.5, 3],raio=2)